In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import dolfinx as dfx

from dolfinx.fem.petsc import NonlinearProblem
from dolfinx.nls.petsc import NewtonSolver

from matplotlib import pyplot as plt

from mpi4py import MPI

import numpy as np

from petsc4py import PETSc

import pyvista

import random

import ufl

from fenicsx_utils import evaluation_points_and_cells, get_mesh_spacing

# Below is a workaround to get the pyvista viz. running.
# https://github.com/pyvista/pyvista/issues/4776 (accessed: 2024/01/16)
from trame.app import get_server

CLIENT_TYPE = get_server().client_type

if CLIENT_TYPE == 'vue2':
# if True:  # <- Strangely, the workaround does not work. Force vuetify2.
    from trame.widgets import vuetify2
else:
    from trame.widgets import vuetify3 as vuetify

In [ ]:
comm_world = MPI.COMM_WORLD

In [ ]:
# Set up the mesh

n_elem = 64

mesh = dfx.mesh.create_unit_interval(comm_world, n_elem)

dx_cell = get_mesh_spacing(mesh)

print(f"Cell spacing: h = {dx_cell}")

# For later plotting use
x = np.linspace(0, 1, 101)
points_on_proc, cells = evaluation_points_and_cells(mesh, x)

In [ ]:
elem1 = ufl.FiniteElement("Lagrange", mesh.ufl_cell(), 1)

V = dfx.fem.FunctionSpace(mesh, elem1 * elem1)  # A mixed two-component function space

In [ ]:
# The mixed-element functions
u = dfx.fem.Function(V)
u0 = dfx.fem.Function(V)

In [ ]:
# Compute the chemical potential df/dc
free_energy = lambda u: 0.25 * (u**2 -1)**2 - 0.25

fig, ax = plt.subplots()

c_plot = np.linspace(-2, 2, 200)

ax.plot(c_plot, free_energy(c_plot))

plt.show()

In [ ]:
dt = dfx.fem.Constant(mesh, dx_cell * 0.1)

In [ ]:
# The variational form
# --------------------
from cahn_hilliard_utils import cahn_hilliard_form

F = cahn_hilliard_form(u, u0, dt, free_energy=free_energy, theta=0.75, I_charge=0., lam=1.0)

In [ ]:
# Initial data
# ------------

u_ini = dfx.fem.Function(V)

# Random
u_ini.sub(0).interpolate(lambda x: 0.01 * np.random.randn(*x[0].shape))

# Zero-mean
u_ini.sub(0).interpolate(lambda x: np.cos(np.pi * x[0]))

# Constant
# u_ini.sub(0).interpolate(lambda x: 1e-3 * np.ones_like(x[0]))
u_ini.sub(1).interpolate(lambda x: np.zeros_like(x[0]))

u_ini.x.scatter_forward()

u_ini_vals = u_ini.sub(0).eval(points_on_proc, cells)

plt.figure()

plt.plot(x, u_ini_vals)

plt.show()

In [ ]:
problem = NonlinearProblem(F, u)

solver = NewtonSolver(comm_world, problem)

solver.convergence_criterion = "incremental"
solver.rtol = 1e-6

# # We can customize the linear solver used inside the NewtonSolver by
# # modifying the PETSc options
ksp = solver.krylov_solver
opts = PETSc.Options()  # type: ignore
option_prefix = ksp.getOptionsPrefix()
opts[f"{option_prefix}ksp_type"] = "preonly"
opts[f"{option_prefix}pc_type"] = "lu"
ksp.setFromOptions()

In [ ]:
# The time loop
t = 0.

T = 10.  # ending time

u.interpolate(u_ini)

u_out = []
t_out = []

n_out = 11
it_out = 0

# Write the initial data.
u_out.append(u.sub(0).eval(points_on_proc, cells))
t_out.append(t)
it_out += 1

while t < T:
# for it in range(1):

    u0.x.array[:] = u.x.array[:]

    if float(dt) < 1e-6:

        u_out.append(u.sub(0).eval(points_on_proc, cells))
        t_out.append(t)

        raise RuntimeError(f"Timestep too small (dt={dt.value})!")

    try:
        iterations, success = solver.solve(u)
    except RuntimeError as e:
        print(e)

        # reset and continue with smaller time step.
        u.x.array[:] = u0.x.array[:]

        dt.value *= 0.5

        print(f"Decrease timestep to dt={dt.value:1.3e}")

        continue

    t += float(dt)

    if t > it_out / n_out * T:

        print(f">>> output #{it_out:>4}")

        u_out.append(u.sub(0).eval(points_on_proc, cells))
        t_out.append(t)
        it_out += 1

    dt.value *= 1.01

    print(f"t = {t:1.6f} : dt = {dt.value:1.3e}, its = {iterations}")

In [ ]:
fig, ax = plt.subplots()

for it_out, (u, t) in enumerate(zip(u_out, t_out)):

    color = (it_out / len(t_out), 0, 0)

    ax.plot(x, u, color=color)

plt.show()